In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('logreg').getOrCreate()

df  = spark.read.csv('/FileStore/tables/titanic.csv', inferSchema=True, header=True)
df.printSchema()

root
-- PassengerId: integer (nullable = true)
-- Survived: integer (nullable = true)
-- Pclass: integer (nullable = true)
-- Name: string (nullable = true)
-- Sex: string (nullable = true)
-- Age: double (nullable = true)
-- SibSp: integer (nullable = true)
-- Parch: integer (nullable = true)
-- Ticket: string (nullable = true)
-- Fare: double (nullable = true)
-- Cabin: string (nullable = true)
-- Embarked: string (nullable = true)

In [2]:
cols = df.select(['Survived','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked'])

In [3]:
# drop missing data
final_data = cols.na.drop()

In [4]:
from pyspark.ml.feature import VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer

In [5]:
# String Index
gender_indexer = StringIndexer(inputCol='Sex', outputCol='SexIndex')

# One Hot Encoding
gender_encoder = OneHotEncoder(inputCol='SexIndex', outputCol='SexVec')

In [6]:
embark_indexer = StringIndexer(inputCol='Embarked', outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex', outputCol='EmbarkVec')

In [7]:
assembler = VectorAssembler(inputCols=['Pclass', 'SexVec', 'EmbarkVec', 'Age', 'SibSp', 'Parch', 'Fare'], 
                            outputCol='features')

In [8]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

log_reg_titanic = LogisticRegression(featuresCol = 'features', labelCol='Survived')
pipeline = Pipeline(stages=[gender_indexer, embark_indexer, gender_encoder, embark_encoder, assembler, log_reg_titanic])

In [9]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])
fit_model = pipeline.fit(train_data)
result = fit_model.transform(test_data)

In [10]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Survived')
result.select('Survived', 'prediction').show()

+--------+----------+
Survived|prediction|
+--------+----------+
 0| 1.0|
 0| 1.0|
 0| 1.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 1.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 1.0|
 0| 1.0|
+--------+----------+
only showing top 20 rows

In [11]:
AUC = my_eval.evaluate(result)
AUC

Out[51]: 0.8166666666666667